In [1]:
import os
import shutil
import json

import numpy as np
import nibabel as nib

from utils import read_tracking, write_tracking

In [2]:
with open("sweep_intervals.json", "r", encoding="utf-8") as f:
    sweep_intervals = json.load(f)
print(sweep_intervals)

{'leg': [361, 822, 1151, 1441, 1714], 'liver': [200, 400, 600, 800, 1000, 1200], 'spine': [152, 305, 456, 608, 756, 902, 1052, 1200, 1341]}


In [3]:
leg_paths = {
    "data": "leg/leg_data_refined.nii.gz",
    "conf": "leg/leg_conf_refined.nii.gz",
    "tracking": "leg/leg_tracking.csv",
}

liver_paths = {
    "data": "liver/liver_all_sweeps_images_refined.nii.gz",
    "conf": "liver/liver_all_sweeps_confidence_maps_refined.nii.gz",
    "tracking": "liver/liver_all_sweeps_tracking.csv",
}

spine_paths = {
    "data": "spine/spine_all_sweeps_refined.nii.gz",
    "conf": "spine/spine_all_sweeps_conf_refined.nii.gz",
    "tracking": "spine/spine_all_sweeps_tracking.csv",
}

In [4]:
# Create output directory "views" for each organ

for organ in ["leg", "liver", "spine"]:
    path = os.path.join(organ, "views")

    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)

In [5]:
# Load data

data = {
    "leg": {
        "data": nib.load(leg_paths["data"]).get_fdata(),
        "conf": nib.load(leg_paths["conf"]).get_fdata(),
        "tracking": read_tracking(leg_paths["tracking"]),
        "header": nib.load(leg_paths["data"]).header
    },
    "liver": {
        "data": nib.load(liver_paths["data"]).get_fdata(),
        "conf": nib.load(liver_paths["conf"]).get_fdata(),
        "tracking": read_tracking(liver_paths["tracking"]),
        "header": nib.load(liver_paths["data"]).header
    },
    "spine": {
        "data": nib.load(spine_paths["data"]).get_fdata(),
        "conf": nib.load(spine_paths["conf"]).get_fdata(),
        "tracking": read_tracking(spine_paths["tracking"]),
        "header": nib.load(spine_paths["data"]).header
    },
}

In [7]:
# Print data shapes

for organ in ["leg", "liver", "spine"]:
    print(organ)
    for key in data[organ]:
        if key != "header":
            print(key, data[organ][key].shape)
        else:
            print(key, data[organ][key])

leg
data (221, 537, 1714)
conf (221, 537, 1714)
tracking (1714, 4, 4)
header <class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 0
dim             : [   3  221  537 1714    1    1    1    1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : uint8
bitpix          : 8
slice_start     : 0
pixdim          : [1.       0.172336 0.17278  1.       1.       1.       1.       1.      ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : unknown
sform_code      : aligned
quatern_b       : 0.0
quatern_c       : 0.0
quatern_d    

In [10]:
# Split the data into views

for organ in ["leg", "liver", "spine"]:

    intervals = [0] + sweep_intervals[organ]

    for i in range(len(intervals) - 1):

        view_data = data[organ]["data"][:, :, intervals[i] : intervals[i + 1]]
        view_conf = data[organ]["conf"][:, :, intervals[i] : intervals[i + 1]]
        view_tracking = data[organ]["tracking"][intervals[i] : intervals[i + 1]]

        # Save the view data

        view_data_path = os.path.join(organ, "views", f"{i}.nii.gz")
        # Use the same header as the original data
        view_data_nii = nib.Nifti1Image(view_data, affine=np.eye(4), header=data[organ]["header"])
        nib.save(view_data_nii, view_data_path)

        # Save the view confidence

        view_conf_path = os.path.join(organ, "views", f"{i}_conf.nii.gz")
        view_conf_nii = nib.Nifti1Image(view_conf, affine=np.eye(4), header=data[organ]["header"])
        nib.save(view_conf_nii, view_conf_path)

        # Save the view tracking
        
        write_tracking(os.path.join(organ, "views", f"{i}_tracking.csv"), view_tracking)

        # Print the view data shapes

        print(organ, i)
        print(view_data.shape)
        print(view_conf.shape)
        print(view_tracking.shape)
        
        



leg 0
(221, 537, 361)
(221, 537, 361)
(361, 4, 4)
leg 1
(221, 537, 461)
(221, 537, 461)
(461, 4, 4)
leg 2
(221, 537, 329)
(221, 537, 329)
(329, 4, 4)
leg 3
(221, 537, 290)
(221, 537, 290)
(290, 4, 4)
leg 4
(221, 537, 273)
(221, 537, 273)
(273, 4, 4)
liver 0
(256, 512, 200)
(256, 512, 200)
(200, 4, 4)
liver 1
(256, 512, 200)
(256, 512, 200)
(200, 4, 4)
liver 2
(256, 512, 200)
(256, 512, 200)
(200, 4, 4)
liver 3
(256, 512, 200)
(256, 512, 200)
(200, 4, 4)
liver 4
(256, 512, 200)
(256, 512, 200)
(200, 4, 4)
liver 5
(256, 512, 200)
(256, 512, 200)
(200, 4, 4)
spine 0
(164, 1300, 152)
(164, 1300, 152)
(152, 4, 4)
spine 1
(164, 1300, 153)
(164, 1300, 153)
(153, 4, 4)
spine 2
(164, 1300, 151)
(164, 1300, 151)
(151, 4, 4)
spine 3
(164, 1300, 152)
(164, 1300, 152)
(152, 4, 4)
spine 4
(164, 1300, 148)
(164, 1300, 148)
(148, 4, 4)
spine 5
(164, 1300, 146)
(164, 1300, 146)
(146, 4, 4)
spine 6
(164, 1300, 150)
(164, 1300, 150)
(150, 4, 4)
spine 7
(164, 1300, 148)
(164, 1300, 148)
(148, 4, 4)
spine 